## 9.1 语言模型的背景知识

###  9.1.1 语言模型的简介

In [ ]:
import tensorflow as tf

In [3]:
word_labels = tf.constant([2,0])

In [4]:
predict_logits = tf.constant([[2.0,-1.0,3.0],[1.0,0.0,-0.5]])

In [5]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=word_labels, logits = predict_logits)

In [6]:
sess = tf.Session()
sess.run(loss)

array([0.32656264, 0.4643688 ], dtype=float32)

In [7]:
# softmax_cross_entropy_with_logits需要将预测的目标以概率分布的形式给出
word_prob_distribution = tf.constant([[0.0,0.0,1.0],[1.0,0.0,0.0]])
loss = tf.nn.softmax_cross_entropy_with_logits(labels = word_prob_distribution, logits = predict_logits)
sess.run(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



array([0.32656264, 0.4643688 ], dtype=float32)

##  9.2 神经语言模型

In [8]:
tf.__version__

'1.13.1'

In [9]:
import codecs
import collections
from operator import itemgetter

In [12]:
RAW_DATA = "./data/simple-examples/data/ptb.train.txt"
VOCAB_OUTPUT = "./data/afterPro/ptb.vocab"

counter = collections.Counter()

In [13]:
with codecs.open(RAW_DATA, "r", "utf-8") as f:
    for line in f:
        for word in line.strip().split():
            counter[word]+=1

In [15]:
type(counter)

collections.Counter

In [18]:
type(counter.items())

dict_items

In [ ]:
sorted_word_to_cnt = sorted(counter.items(),
                           key=itemgetter(1),
                           reverse = True)

In [20]:
type(sorted_word_to_cnt)

list

In [22]:
sorted_word_to_cnt[10]

('for', 8927)

In [17]:
sorted_words = [x[0] for x in sorted_word_to_cnt]

sorted_words = ["<eos>"] + sorted_words

with codecs.open(VOCAB_OUTPUT, 'w', 'utf-8') as file_output:
    for word in sorted_words:
        file_output.write(word+"\n")

######  在确定词汇表之后，再将训练文件、测试文件等都根据词汇文件转化为单词编号。每个单词文件的编号就是它在词汇文件中的行号

In [37]:
import codecs
import sys

RAW_DATA = "./data/simple-examples/data/ptb.valid.txt"#train test valid
VOCAB = "./data/afterPro/ptb.vocab"
OUTPUT_DATA = "./data/afterPro/ptb.valid"

In [38]:
VOCAB

'./data/afterPro/ptb.vocab'

In [39]:
input_vocab_file = VOCAB
with codecs.open(input_vocab_file, "r", "utf-8") as f_vocab:
    vocab = [w.strip() for w in f_vocab.readlines()]

In [40]:
vocab[:5]

['<eos>', 'the', '<unk>', 'N', 'of']

In [41]:
word_to_id = {k:v for (k,v) in zip(vocab, range(len(vocab)))}

In [42]:
def get_id(word):
    return word_to_id[word] if word in word_to_id else word_to_id["<unk>"]

In [43]:
fin = codecs.open(RAW_DATA, "r", "utf-8")
fout = codecs.open(OUTPUT_DATA, "w", 'utf-8')
for line in fin:
    words = line.strip().split()+["<eos>"]
    out_line = ' '.join([str(get_id(w))  for w in words]) + '\n'
    fout.write(out_line)
fin.close()
fout.close()

###  9.2.2 PTB数据的batching方法

为了利用计算时的并行能力，我们希望每一次计算对多个句子进行并行处理，同时要保证batch之间的上下文连续。
解决方案是先将整个文档切分成若干连续段落，再让batch中每一个位置负责其中一段

In [37]:
import numpy as np
import tensorflow as tf

TRAIN_DATA = "./data/afterPro/ptb.train"
TRAIN_BATCH_SIZE = 20
TRAIN_NUM_STEP = 35

In [38]:
#  从文件中读取数据，并返回包含单词编号的数据组
def read_data(file_path):
    with open(file_path, "r") as fin:
        id_string = ' '.join([line.strip() for line in fin.readlines()])
    id_list = [int(w) for w in id_string.split()]
    return id_list

In [42]:
def make_batches(id_list, batch_size,num_step):
    # 计算总的batch数量。每个batch包含的单词数量是batch_size*num_step
    num_batches = (len(id_list) -1) // (batch_size*num_step)
    
    #将数据整理为一个二维数组，维度：[batch_size, num_batches*num_step]
    data = np.array(id_list[:num_batches*batch_size*num_step])
    data = np.reshape(data, [batch_size, num_batches*num_step])
    # 沿着第二个维度，将数据切成num_batches个batch，存入一个数组
    data_batches = np.split(data, num_batches, axis=1)
    
    # 重复上述操作，但是每个位置向右移动一位。
    # 这里得到RNN每一步输出所需要预测的下一个单词
    label = np.array(id_list[1:num_batches*batch_size*num_step+1])
    label = np.reshape(label, [batch_size,num_batches*num_step])
    label = np.split(data,num_batches, axis=1)
    # 返回一个长度为num_batches的数组，其中每一项包含一个data矩阵和一个label矩阵
    return list(zip(data_batches, label_batches))

###  9.2.3 基于循环神经网络的神经语言模型

#### 词向量层

- 作用：降低输入维度；增加语义信息

In [2]:
import tensorflow as tf

embedding = tf.get_variable("embedding", [VOCAB_SIZE, EMB_SIZE])
input_embedding = tf.nn.embedding_lookup(emdedding, input_data)

- 输入的input_data的维度：batch_size \* num_steps
- 输出的input_embedding的维度：batch_size \* num_steps\*EMB_SIZE

#### softmax层
- 将神经网络的输出转化为一个单词表中每个单词出现的概率

####  通过共享参数减少参数量

### 完整的训练程序

####  1. 设置参数

In [49]:
# coding : utf-8
import numpy as np
import tensorflow as tf

TRAIN_DATA  = "./data/afterPro/ptb.train"
EVAL_DATA   = "./data/afterPro/ptb.valid"
TEST_DATA   = "./data/afterPro/ptb.test"
HIDDEN_SIZE = 300                 # 隐藏层规模。
NUM_LAYERS = 2                    # 深层循环神经网络中LSTM结构的层数。
VOCAB_SIZE = 10000                # 词典规模。
TRAIN_BATCH_SIZE = 20             # 训练数据batch的大小。
TRAIN_NUM_STEP = 35               # 训练数据截断长度。

EVAL_BATCH_SIZE = 1               # 测试数据batch的大小。
EVAL_NUM_STEP = 1                 # 测试数据截断长度。
NUM_EPOCH = 5                     # 使用训练数据的轮数。
LSTM_KEEP_PROB = 0.9              # LSTM节点不被dropout的概率。
EMBEDDING_KEEP_PROB = 0.9         # 词向量不被dropout的概率。
MAX_GRAD_NORM = 5                 # 用于控制梯度膨胀的梯度大小上限。
SHARE_EMB_AND_SOFTMAX = True      # 在Softmax层和词向量层之间共享参数。

####  2. 定义模型

In [50]:
# 通过一个PTBModel类来描述模型，这样方便维护循环神经网络中的状态。
class PTBModel(object):
    def __init__(self, is_training, batch_size, num_steps):
        # 记录使用的batch大小和截断长度。
        self.batch_size = batch_size
        self.num_steps = num_steps
        
        # 定义每一步的输入和预期输出。两者的维度都是[batch_size, num_steps]。
        self.input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
        self.targets = tf.placeholder(tf.int32, [batch_size, num_steps])
        
        # 定义使用LSTM结构为循环体结构且使用dropout的深层循环神经网络。
        dropout_keep_prob = LSTM_KEEP_PROB if is_training else 1.0
        lstm_cells = [
            tf.nn.rnn_cell.DropoutWrapper(
                tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE),
                output_keep_prob=dropout_keep_prob)
            for _ in range(NUM_LAYERS)]     
        cell = tf.nn.rnn_cell.MultiRNNCell(lstm_cells)            
        
        # 初始化最初的状态，即全零的向量。这个量只在每个epoch初始化第一个batch
        # 时使用。
        self.initial_state = cell.zero_state(batch_size, tf.float32)

        # 定义单词的词向量矩阵。
        embedding = tf.get_variable("embedding", [VOCAB_SIZE, HIDDEN_SIZE])
        
        # 将输入单词转化为词向量。
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)
        
        # 只在训练时使用dropout。
        if is_training:
            inputs = tf.nn.dropout(inputs, EMBEDDING_KEEP_PROB)
 
        # 定义输出列表。在这里先将不同时刻LSTM结构的输出收集起来，再一起提供给
        # softmax层。
        outputs = []
        state = self.initial_state
        with tf.variable_scope("RNN"):
            for time_step in range(num_steps):
                if time_step > 0: tf.get_variable_scope().reuse_variables()
                    #cell的输入：batch_size *EMB_SIZE
                    #状态维度：?
                    #输出维度：?
                cell_output, state = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output) 
        # 把输出队列展开成[batch, hidden_size*num_steps]的形状，然后再
        # reshape成[batch*numsteps, hidden_size]的形状。
        output = tf.reshape(tf.concat(outputs, 1), [-1, HIDDEN_SIZE])
 
        # Softmax层：将RNN在每个位置上的输出转化为各个单词的logits。
        if SHARE_EMB_AND_SOFTMAX:
            weight = tf.transpose(embedding)
        else:
            weight = tf.get_variable("weight", [HIDDEN_SIZE, VOCAB_SIZE])
        bias = tf.get_variable("bias", [VOCAB_SIZE])
        logits = tf.matmul(output, weight) + bias
        
        # 定义交叉熵损失函数和平均损失。
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.reshape(self.targets, [-1]),
            logits=logits)
        self.cost = tf.reduce_sum(loss) / batch_size
        self.final_state = state
        
        # 只在训练模型时定义反向传播操作。
        if not is_training: return

        trainable_variables = tf.trainable_variables()
        # 控制梯度大小，定义优化方法和训练步骤。
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(self.cost, trainable_variables), MAX_GRAD_NORM)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0)
        self.train_op = optimizer.apply_gradients(
            zip(grads, trainable_variables)) 

#### 3.定义数据和训练过程。

In [51]:
# 使用给定的模型model在数据data上运行train_op并返回在全部数据上的perplexity值。
def run_epoch(session, model, batches, train_op, output_log, step):
    # 计算平均perplexity的辅助变量。
    total_costs = 0.0
    iters = 0
    state = session.run(model.initial_state) 
    # 训练一个epoch。
    for x, y in batches:
        # 在当前batch上运行train_op并计算损失值。交叉熵损失函数计算的就是下一个单
        # 词为给定单词的概率。
        cost, state, _ = session.run(
             [model.cost, model.final_state, train_op],
             {model.input_data: x, model.targets: y,
              model.initial_state: state})
        total_costs += cost
        iters += model.num_steps

        # 只有在训练时输出日志。
        if output_log and step % 100 == 0:
            print("After %d steps, perplexity is %.3f" % (
                  step, np.exp(total_costs / iters)))
        step += 1

    # 返回给定模型在给定数据上的perplexity值。
    return step, np.exp(total_costs / iters)


# 从文件中读取数据，并返回包含单词编号的数组。
def read_data(file_path):
    with open(file_path, "r") as fin:
        # 将整个文档读进一个长字符串。
        id_string = ' '.join([line.strip() for line in fin.readlines()])
    id_list = [int(w) for w in id_string.split()]  # 将读取的单词编号转为整数
    return id_list


def make_batches(id_list, batch_size, num_step):
    # 计算总的batch数量。每个batch包含的单词数量是batch_size * num_step。
    num_batches = (len(id_list) - 1) // (batch_size * num_step)

    # 如9-4图所示，将数据整理成一个维度为[batch_size, num_batches * num_step]
    # 的二维数组。
    data = np.array(id_list[: num_batches * batch_size * num_step])
    data = np.reshape(data, [batch_size, num_batches * num_step])
    # 沿着第二个维度将数据切分成num_batches个batch，存入一个数组。
    data_batches = np.split(data, num_batches, axis=1)

    # 重复上述操作，但是每个位置向右移动一位。这里得到的是RNN每一步输出所需要预测的
    # 下一个单词。
    label = np.array(id_list[1 : num_batches * batch_size * num_step + 1]) 
    label = np.reshape(label, [batch_size, num_batches * num_step])
    label_batches = np.split(label, num_batches, axis=1)  
    # 返回一个长度为num_batches的数组，其中每一项包括一个data矩阵和一个label矩阵。
    return list(zip(data_batches, label_batches)) 


#### 4.主函数

In [ ]:
def main():
    # 定义初始化函数。
    initializer = tf.random_uniform_initializer(-0.05, 0.05)
    
    # 定义训练用的循环神经网络模型。
    with tf.variable_scope("language_model", 
                           reuse=True, initializer=initializer): #------------------------------------
        train_model = PTBModel(True, TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)

    # 定义测试用的循环神经网络模型。它与train_model共用参数，但是没有dropout。
    with tf.variable_scope("language_model",
                           reuse=True, initializer=initializer):
        eval_model = PTBModel(False, EVAL_BATCH_SIZE, EVAL_NUM_STEP)

    # 训练模型。
    with tf.Session() as session:
        tf.global_variables_initializer().run()
        train_batches = make_batches(
            read_data(TRAIN_DATA), TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)
        eval_batches = make_batches(
            read_data(EVAL_DATA), EVAL_BATCH_SIZE, EVAL_NUM_STEP)
        test_batches = make_batches(
            read_data(TEST_DATA), EVAL_BATCH_SIZE, EVAL_NUM_STEP)

        step = 0
        for i in range(NUM_EPOCH):
            print("In iteration: %d" % (i + 1))
            step, train_pplx = run_epoch(session, train_model, train_batches,
                                         train_model.train_op, True, step)
            print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_pplx))

            _, eval_pplx = run_epoch(session, eval_model, eval_batches,
                                     tf.no_op(), False, 0)
            print("Epoch: %d Eval Perplexity: %.3f" % (i + 1, eval_pplx))

        _, test_pplx = run_epoch(session, eval_model, test_batches,
                                 tf.no_op(), False, 0)
        print("Test Perplexity: %.3f" % test_pplx)

if __name__ == "__main__":
    main()